In [20]:
import collections
import datetime
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi
import seaborn as sns
import tensorflow as tf
from IPython import display
from matplotlib import pyplot as plt
from typing import Dict, List, Optional, Sequence, Tuple
import os

In [2]:
import fluidsynth

In [3]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Sampling rate for audio playback
_SAMPLING_RATE = 16000

In [23]:
#import all the file names
filenames = glob.glob(str("MIDI/pop/**/*.mid"))
# filenames = glob.glob(str("/home/jayden/projects/project_lyrics/MIDI/pop/**/*.mid"))
print('Number of files: ', len(filenames))
sample_file = filenames[5]
print(sample_file)
pm = pretty_midi.PrettyMIDI(sample_file)

Number of files:  2898
MIDI/pop/Romantics/GangnamStyle.mid


In [18]:
#악기 종류 찾아내기
print('Number of instruments:', len(pm.instruments))
instrument = pm.instruments[0]
instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
print('Instrument name:', instrument_name)

for num in range(len(pm.instruments)):
    print('All instruments:' , pretty_midi.program_to_instrument_name(pm.instruments[num].program))

Number of instruments: 12
Instrument name: Celesta
All instruments: Celesta
All instruments: Lead 2 (sawtooth)
All instruments: Lead 1 (square)
All instruments: Lead 1 (square)
All instruments: Synth Brass 1
All instruments: Applause
All instruments: String Ensemble 1
All instruments: Rock Organ
All instruments: Melodic Tom
All instruments: Reverse Cymbal
All instruments: Lead 1 (square)
All instruments: Distortion Guitar


In [22]:
#제일 많이 쓰이는 악기 20가지만 뽑아보기 [이유는 보통 같은 장르면 중복되는것이 많을것이라 추축]

def count_instruments(file_list):
    instrument_counts = {i: 0 for i in range(128)}
    files_to_delete = []  # List to store files with errors
    for file_name in file_list:
        try:
            pm = pretty_midi.PrettyMIDI(file_name)
            for instrument in pm.instruments:
                for note in instrument.notes:
                    instrument_counts[instrument.program] += 1
        except Exception as e:
            print(f"Error processing file {file_name}: {e}")
            files_to_delete.append(file_name)

    # Delete the files with errors
    for file_to_delete in files_to_delete:
        os.remove(file_to_delete)
        
    return instrument_counts


instrument_counts = count_instruments(filenames)
for program, count in instrument_counts.items():
    print(f"Instrument {program}: {count} occurrences")

    

/home/jayden/miniconda3/envs/aimusic/lib/python3.11/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Error processing file MIDI/pop/5th Dimension/OneLessBellToAnswere.mid: no MTrk header at start of track
Error processing file MIDI/pop/Bee Gee/GreaseMegamix .mid: data byte must be in range 0..127
Error processing file MIDI/pop/Newsboys/PartyInTheUSA.mid: data byte must be in range 0..127
Error processing file MIDI/pop/Gary Wright/DarnThatDream.mid: data byte must be in range 0..127
Error processing file MIDI/pop/Brandy/CatLikeThief.mid: MThd not found. Probably not a MIDI file
Error processing file MIDI/pop/Ricky Martin/SolsburyHill.mid: data byte must be in range 0..127
Error processing file MIDI/pop/Haddaway/AnythingForYou.mid: MThd not found. Probably not a MIDI file
Error processing file MIDI/pop/Haddaway/BreakingUpIsHardToDo.mid: MThd not found. Probably not a MIDI file
Error processing file MIDI/pop/Haddaway/MiTierra.mid: MThd not found. Probably not a MIDI file
Error processing file MIDI/pop/Haddaway/ImNotGivinYouUp.mid: MThd not found. Probably not a MIDI file
Error processing

In [26]:
def count_instruments(file_list):
    instrument_counts = {}
    files_to_delete = []  # List to store files with errors
    
    for file_name in file_list:
        try:
            pm = pretty_midi.PrettyMIDI(file_name)
            for instrument in pm.instruments:
                instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
                if instrument_name not in instrument_counts:
                    instrument_counts[instrument_name] = 0
                instrument_counts[instrument_name] += 1
        except Exception as e:
            print(f"Error processing file {file_name}: {e}")
            # Add the file to the list of files to delete
            files_to_delete.append(file_name)
    
    # Delete the files with errors
    for file_to_delete in files_to_delete:
        os.remove(file_to_delete)
    
    # Convert the dictionary to a DataFrame
    df = pd.DataFrame.from_dict(instrument_counts, orient='index', columns=['Count'])
    return df

# Example usage:
instrument_counts_df = count_instruments(filenames)
print(instrument_counts_df)

/home/jayden/miniconda3/envs/aimusic/lib/python3.11/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


work flow:
get midi file
verify can extract notes of instruments -done   -> can train on single instrument.


verify if all notes of the instruments can produce a single value or if all instruments need to be trained individually


how to give new input so it will start producing music


extract all notes of midi file
